# Keras - Transfer Learning - Licence Plates Recognition

In [1]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import shutil
import itertools
import xml.etree.ElementTree as xet

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import pandas as pd

In [4]:
print("TF version:", tf.__version__)
#print("Hub version:", hub.__version__)

TF version: 2.8.0


In [5]:
phisical_devices = tf.config.list_physical_devices('GPU')
print("GPU is", "available" if len(phisical_devices) > 0 else "NOT AVAILABLE")

GPU is NOT AVAILABLE


2022-03-28 22:43:14.542324: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-28 22:43:14.585544: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2022-03-28 22:43:14.585560: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model = keras.applications.vgg16.VGG16()

2022-03-28 22:43:14.604524: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model_modified = Sequential(name='vgg16_modified')
for layer in model.layers[:-1] :
    layer.trainable = False
    model_modified.add(layer)

In [8]:
model_modified.add(Dense(units=4, activation='softmax'))

In [9]:
model_modified.summary()

Model: "vgg16_modified"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)    

In [10]:
model_modified.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

## Preparing Data

In [11]:
train_path = 'data/licence_plates/train'
valid_path = 'data/licence_plates/valid'
test_path = 'data/licence_plates/test'

In [12]:
train_images_paths = glob.glob(train_path + "/*.jpeg")
train_labels_paths = glob.glob(train_path + "/*.xml")

In [13]:
print(train_images_paths[:3])
print(train_labels_paths[:3])

['data/licence_plates/train/N123.jpeg', 'data/licence_plates/train/N179.jpeg', 'data/licence_plates/train/N85.jpeg']
['data/licence_plates/train/N152.xml', 'data/licence_plates/train/N49.xml', 'data/licence_plates/train/N130.xml']


### Extracting labes from xml files

In [14]:
def get_bounding_box_from_xml_file(path):
    parsed_xml_file = xet.parse(path)
    root_element = parsed_xml_file.getroot()
    object_element = root_element.find('object')
    bndbox_element = object_element.find('bndbox')
    
    xmin = int(bndbox_element.find('xmin').text)
    xmax = int(bndbox_element.find('xmax').text)
    ymin = int(bndbox_element.find('ymin').text)
    ymax = int(bndbox_element.find('ymax').text)
    
    return (xmin, xmax, ymin, ymax)

def get_img_filename_from_xml_file(path):
    parsed_xml_file = xet.parse(path)
    root_element = parsed_xml_file.getroot()
    filename = root_element.find('filename').text
    return filename

In [15]:
# just to test the above functions
out = get_bounding_box_from_xml_file(train_labels_paths[0])
out

(265, 725, 247, 347)

In [16]:
# first create an empty map
labels_map = {'file_name':[], 'xmin':[], 'xmax':[], 'ymin':[], 'ymax':[]}

# load data into the map

i = 0
for label_file in train_labels_paths:
    
    i+=1
    bndbox = get_bounding_box_from_xml_file(label_file)
    filename = get_img_filename_from_xml_file(label_file)
    
    # print(filename, bndbox)
    
    labels_map.get('file_name').append(filename)
    labels_map.get('xmin').append(bndbox[0])
    labels_map.get('xmax').append(bndbox[1])
    labels_map.get('ymin').append(bndbox[2])
    labels_map.get('ymax').append(bndbox[3])

In [17]:
df = pd.DataFrame(labels_map)
df.head()

,file_name,xmin,xmax,ymin,ymax
0,N152.jpeg,265,725,247,347
1,N49.jpeg,196,352,135,173
2,N130.jpeg,419,513,173,213
3,N120.jpeg,647,736,332,387
4,N154.jpeg,106,245,157,196


In [18]:
# Add image file to the data frame
file_names = df['file_name']
new_image_paths = []
for file_name in file_names:
    new_image_paths.append(train_path + file_name)


df['img_path'] = new_image_paths
df.head()

,file_name,xmin,xmax,ymin,ymax,img_path
0,N152.jpeg,265,725,247,347,data/licence_plates/trainN152.jpeg
1,N49.jpeg,196,352,135,173,data/licence_plates/trainN49.jpeg
2,N130.jpeg,419,513,173,213,data/licence_plates/trainN130.jpeg
3,N120.jpeg,647,736,332,387,data/licence_plates/trainN120.jpeg
4,N154.jpeg,106,245,157,196,data/licence_plates/trainN154.jpeg


In [19]:
def preprocess_image(image_path,resize=False):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    if resize:
        img = cv2.resize(img, (224,224))
    return img


In [19]:
train_data_gen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)


In [ ]:
train_data_gen.flow(x=, target_size=(224,224), classes=['cats', 'dogs'])